In [1]:
!pip install grpcio grpcio-tools utils


[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
grpc_host = 'modelmesh-serving'
grpc_port = 8033
model_name = 'v33'

In [3]:
import sys
sys.path.append('./utils')

# grpc_predict_v2_pb2 and grpc_predict_v2_pb2_grpc were created from grpc_predict_v2.proto using protoc
import grpc
import utils.grpc_predict_v2_pb2 as grpc_predict_v2_pb2
import utils.grpc_predict_v2_pb2_grpc as grpc_predict_v2_pb2_grpc


channel = grpc.insecure_channel(f"{grpc_host}:{grpc_port}")
stub = grpc_predict_v2_pb2_grpc.GRPCInferenceServiceStub(channel)

request = grpc_predict_v2_pb2.ModelMetadataRequest(name=model_name)
response = stub.ModelMetadata(request)
response

name: "v33__isvc-d2bd1d1553"
versions: "1"
platform: "onnxruntime_onnx"
inputs {
  name: "input.1"
  datatype: "FP32"
  shape: 1
  shape: 1
  shape: 160
  shape: 160
  shape: 160
}
outputs {
  name: "252"
  datatype: "FP32"
  shape: 1
  shape: 2
  shape: 160
  shape: 160
  shape: 160
}

In [4]:
import numpy as np

def grpc_request(data):
    # request content building
    inputs = []
    inputs.append(grpc_predict_v2_pb2.ModelInferRequest().InferInputTensor())
    inputs[0].name = "input.1"
    inputs[0].datatype = "FP32"
    inputs[0].shape.extend([1, 1, 160, 160, 160])
    inputs[0].contents.fp32_contents.extend(data)
    print(inputs)

    # request building
    request = grpc_predict_v2_pb2.ModelInferRequest()
    request.model_name = model_name
    request.inputs.extend(inputs)

    response = stub.ModelInfer(request)
    result_arr = np.frombuffer(response.raw_output_contents[0], dtype=np.float32)
    return result_arr

In [5]:
data =  np.load('data.npy')
prediction = grpc_request(data)

[name: "input.1"
datatype: "FP32"
shape: 1
shape: 1
shape: 160
shape: 160
shape: 160
contents {
}
]


_InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.INTERNAL
	details = "inference.GRPCInferenceService/ModelInfer: INTERNAL: onnx runtime error 2: not enough space: expected 16384000, got 0"
	debug_error_string = "UNKNOWN:Error received from peer  {grpc_message:"inference.GRPCInferenceService/ModelInfer: INTERNAL: onnx runtime error 2: not enough space: expected 16384000, got 0", grpc_status:13, created_time:"2024-04-05T16:36:09.343716658+00:00"}"
>